In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast 

# import os
# if os.name == 'posix' :
#     plt.rc('font', family='AppleGothic')
# else :
#     plt.rc('font', family='Malgun Gothic')
    
plt.rc('axes', unicode_minus=False)

plt.rcParams['font.family'] = 'NanumGothic' 
plt.rcParams['axes.unicode_minus'] = False
import koreanize_matplotlib

## 데이터 불러오기

- 마스터 테이블

In [3]:
master_df = pd.read_csv('./votes/final_retentionchart_8day.csv')
master_df.head()

,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,ret_group,ret_group_over,ret_group_under,gender,group_id,school_id,grade
0,838023,22,5,2023-04-29,2023-05-04,5,2023-04-29,0.515152,50 이상,50 이상,NaN,F,233.0,271.0,1.0
1,838466,127,6,2023-05-02,2023-05-07,5,2023-05-02,0.569444,50 이상,50 이상,NaN,F,321.0,369.0,2.0
2,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,50 이상,50 이상,NaN,F,356.0,271.0,2.0
3,839357,205,8,2023-04-29,2023-05-07,8,2023-04-29,0.515152,50 이상,50 이상,NaN,M,519.0,271.0,1.0
4,840293,170,7,2023-05-03,2023-05-15,12,2023-05-03,0.422977,50 미만,NaN,50 미만,F,838.0,369.0,1.0


In [5]:
master_df['school_id'].value_counts()

school_id
1719.0    538
369.0     535
5372.0    476
271.0     460
352.0     459
5520.0    456
5491.0    455
1478.0    445
4516.0    442
4426.0    437
Name: count, dtype: int64

## 파생변수 추가

- 유저의 지역

In [4]:
school_df = pd.read_parquet('./votes/accounts_school.parquet')
school_df.head()

,id,address,student_count,school_type
0,4,충청북도 충주시,239,H
1,5,충청북도 충주시,160,M
2,6,충청북도 충주시,200,H
3,7,충청북도 충주시,114,H
4,8,충청북도 충주시,139,M


In [ ]:
# 유저 학교의 주소, 유저 학교의 학생 수, 유저 학교의 타입 정보 추가
adress_questioncategory_df = pd.merge(master_df, school_df, left_on='school_id', right_on='id',  how='left')
# 중복되는 학교 id 컬럼 삭제 
adress_questioncategory_df = adress_questioncategory_df.drop(columns=['id'])
adress_questioncategory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4703 entries, 0 to 4702
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          4703 non-null   int64  
 1   total_votes      4703 non-null   int64  
 2   unique_days      4703 non-null   int64  
 3   first_vote       4703 non-null   object 
 4   last_vote        4703 non-null   object 
 5   active_days      4703 non-null   int64  
 6   cohort_day       4703 non-null   object 
 7   retention_day8   4703 non-null   float64
 8   ret_group        4703 non-null   object 
 9   ret_group_over   1841 non-null   object 
 10  ret_group_under  2862 non-null   object 
 11  gender           4703 non-null   object 
 12  group_id         4703 non-null   float64
 13  school_id        4703 non-null   float64
 14  grade            4703 non-null   float64
 15  address          4703 non-null   object 
 16  student_count    4703 non-null   int64  
 17  school_type   

---

- 첫 질문 카테고리

In [10]:
# 질문 유형 정보 테이블
question_category_df = pd.read_parquet('./polls_question_add_category.parquet')
question_category_df.head()

,id,question_text,created_at,category
0,99,가장 신비한 매력이 있는 사람은?,2023-03-31 15:22:53,"[연애, 성격, 감성]"
1,100,"""이 사람으로 한 번 살아보고 싶다"" 하는 사람은?",2023-03-31 15:22:53,"[심리, 상상, 인간관계]"
2,101,미래의 틱톡커는?,2023-03-31 15:22:54,"[미디어, 미래, 상상]"
3,102,여기서 제일 특이한 친구는?,2023-03-31 15:22:54,"[성격, 관찰, 인간관계]"
4,103,가장 지켜주고 싶은 사람은?,2023-03-31 15:22:55,"[심리, 일상, 인간관계]"


In [12]:
# 질문 내용
question_df = pd.read_parquet('./votes/polls_question.parquet')
question_df.head()

,id,question_text,created_at
0,99,가장 신비한 매력이 있는 사람은?,2023-03-31 15:22:53
1,100,"""이 사람으로 한 번 살아보고 싶다"" 하는 사람은?",2023-03-31 15:22:53
2,101,미래의 틱톡커는?,2023-03-31 15:22:54
3,102,여기서 제일 특이한 친구는?,2023-03-31 15:22:54
4,103,가장 지켜주고 싶은 사람은?,2023-03-31 15:22:55


In [14]:
# 질문 정보
questionpiece_df = pd.read_parquet('./votes/polls_questionpiece.parquet')
questionpiece_df.head()

,id,is_voted,created_at,question_id,is_skipped
0,998458,1,2023-04-28 12:27:22,252,0
1,998459,1,2023-04-28 12:27:22,244,0
2,998460,1,2023-04-28 12:27:22,183,0
3,998461,1,2023-04-28 12:27:22,101,0
4,998462,1,2023-04-28 12:27:22,209,0


In [11]:
# 투표 기록 테이블
userquestionrecord_df = pd.read_parquet('votes/accounts_userquestionrecord.parquet') 
userquestionrecord_df.head()

,id,status,created_at,chosen_user_id,question_id,user_id,question_piece_id,has_read,answer_status,answer_updated_at,report_count,opened_times
0,771777,C,2023-04-28 12:27:49,849469,252,849436,998458,0,N,2023-04-28 12:27:49,0,0
1,771800,C,2023-04-28 12:28:02,849446,244,849436,998459,0,N,2023-04-28 12:28:02,0,0
2,771812,C,2023-04-28 12:28:09,849454,183,849436,998460,1,N,2023-04-28 12:28:09,0,0
3,771828,C,2023-04-28 12:28:16,847375,101,849436,998461,0,N,2023-04-28 12:28:16,0,0
4,771851,C,2023-04-28 12:28:26,849477,209,849436,998462,1,N,2023-04-28 12:28:26,0,0


In [25]:
# 유저별 첫번째 질문 id
firstquestion_df = userquestionrecord_df.groupby(['user_id'])['question_id'].first().reset_index()
# 컬럼명 바꾸기
firstquestion_df = firstquestion_df.rename(columns={'question_id': 'firstquestion_id'})
firstquestion_df.head()

,user_id,firstquestion_id
0,838023,121
1,838466,298
2,838642,161
3,839357,179
4,840293,464


In [28]:
# 질문 카테고리와 병합
user_firstquestion_df = pd.merge(firstquestion_df, question_category_df, left_on='firstquestion_id', right_on='id', how='left')
# 필요없는 컬럼 버리기
user_firstquestion_df = user_firstquestion_df.drop(columns=['id','created_at'])
user_firstquestion_df.head()

,user_id,firstquestion_id,question_text,category
0,838023,121,가장 어른스러운 사람은?,"[심리, 성격, 인간관계]"
1,838466,298,옷이 제일 많을 거 같은 사람은?,"[일상, 스타일, 관찰]"
2,838642,161,화목한 가정을 꾸릴거 같은 사람은?,"[일상, 라이프스타일, 인간관계]"
3,839357,179,책을 제일 많이 읽은 것 같은 사람은?,"[일상, 지식, 관찰]"
4,840293,464,뭔가 제대로 한 번 보여줄 것 같은 사람은?,"[성격, 미래, 관찰]"


In [30]:
# 마스터테이블과 합치기
adress_questioncategory_df = pd.merge(adress_questioncategory_df, user_firstquestion_df, on='user_id', how='left')
adress_questioncategory_df.head()

,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,ret_group,ret_group_over,...,gender,group_id,school_id,grade,address,student_count,school_type,firstquestion_id,question_text,category
0,838023,22,5,2023-04-29,2023-05-04,5,2023-04-29,0.515152,50 이상,50 이상,...,F,233.0,271.0,1.0,충청남도 천안시 서북구,491,H,121,가장 어른스러운 사람은?,"[심리, 성격, 인간관계]"
1,838466,127,6,2023-05-02,2023-05-07,5,2023-05-02,0.569444,50 이상,50 이상,...,F,321.0,369.0,2.0,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,"[일상, 스타일, 관찰]"
2,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,50 이상,50 이상,...,F,356.0,271.0,2.0,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,"[일상, 라이프스타일, 인간관계]"
3,839357,205,8,2023-04-29,2023-05-07,8,2023-04-29,0.515152,50 이상,50 이상,...,M,519.0,271.0,1.0,충청남도 천안시 서북구,491,H,179,책을 제일 많이 읽은 것 같은 사람은?,"[일상, 지식, 관찰]"
4,840293,170,7,2023-05-03,2023-05-15,12,2023-05-03,0.422977,50 미만,NaN,...,F,838.0,369.0,1.0,충청남도 아산시,578,H,464,뭔가 제대로 한 번 보여줄 것 같은 사람은?,"[성격, 미래, 관찰]"


---

In [31]:
# 새로 만든 컬럼만 내보내기

# 컬럼명 재정의
adress_questioncategory_df = adress_questioncategory_df.rename(columns={'question_text':'first_question_text', 'category':'first_question_category'})
user_adress_questioncategory_df = adress_questioncategory_df[['user_id', 'address', 'student_count', 'school_type', 'firstquestion_id', 'first_question_text', 'first_question_category']]
user_adress_questioncategory_df.head()

,user_id,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category
0,838023,충청남도 천안시 서북구,491,H,121,가장 어른스러운 사람은?,"[심리, 성격, 인간관계]"
1,838466,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,"[일상, 스타일, 관찰]"
2,838642,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,"[일상, 라이프스타일, 인간관계]"
3,839357,충청남도 천안시 서북구,491,H,179,책을 제일 많이 읽은 것 같은 사람은?,"[일상, 지식, 관찰]"
4,840293,충청남도 아산시,578,H,464,뭔가 제대로 한 번 보여줄 것 같은 사람은?,"[성격, 미래, 관찰]"


In [32]:
# 내보내기
user_adress_questioncategory_df.to_csv('user_adress_questioncategory_df.csv')

- 유저가 참여한 모든 투표의 유형